In [1]:
# import dependencies 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import os
import random

In [2]:
# open browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST chromedriver version for 114.0.5735 google-chrome
Driver [/Users/evanbruno/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


In [3]:
# visit page 
url = 'https://california.reaproject.org/data-tables/gsp-a200n/tools/60001/'
browser.visit(url)

In [6]:
html = browser.html
test = soup(html, 'html.parser')

In [7]:
# obtain counties and IDs

counties = {}

for i in test.find(id='toswoeubfdtqdbfdanvbsfrodnqrtvsf').find('tbody').find_all('tr'):
    for row in i.find_all('td'):
        
        county = row.get_text()
        counties[county] = row.find('input').get('id')

In [8]:
counties

{'Alameda': 'sel_60001',
 'Orange': 'sel_60059',
 'Alpine': 'sel_60003',
 'Placer': 'sel_60061',
 'Amador': 'sel_60005',
 'Plumas': 'sel_60063',
 'Butte': 'sel_60007',
 'Riverside': 'sel_60065',
 'Calaveras': 'sel_60009',
 'Sacramento': 'sel_60067',
 'Colusa': 'sel_60011',
 'San Benito': 'sel_60069',
 'Contra Costa': 'sel_60013',
 'San Bernardino': 'sel_60071',
 'Del Norte': 'sel_60015',
 'San Diego': 'sel_60073',
 'El Dorado': 'sel_60017',
 'San Francisco': 'sel_60075',
 'Fresno': 'sel_60019',
 'San Joaquin': 'sel_60077',
 'Glenn': 'sel_60021',
 'San Luis Obispo': 'sel_60079',
 'Humboldt': 'sel_60023',
 'San Mateo': 'sel_60081',
 'Imperial': 'sel_60025',
 'Santa Barbara': 'sel_60083',
 'Inyo': 'sel_60027',
 'Santa Clara': 'sel_60085',
 'Kern': 'sel_60029',
 'Santa Cruz': 'sel_60087',
 'Kings': 'sel_60031',
 'Shasta': 'sel_60089',
 'Lake': 'sel_60033',
 'Sierra': 'sel_60091',
 'Lassen': 'sel_60035',
 'Siskiyou': 'sel_60093',
 'Los Angeles': 'sel_60037',
 'Solano': 'sel_60095',
 'Madera

In [25]:
html = browser.html
test = soup(html, 'html.parser')

In [10]:
# iterate through table IDs

tables_id = ['page_1', 'page_2', 'page_3']

In [26]:
# iterate through county IDs

for county in counties:
    id_county = counties[county]
    
    #click on each county
    
    browser.find_by_id(id_county).click()
    
    time.sleep(3)
    
    html = browser.html
    test = soup(html, 'html.parser')
    
    data = {}

    for page in tables_id:

        years = []
        
        # get years of each section

        for i in test.find(id=page).find(class_='report_table_data').find('tbody').find('tr'):
            pl = i.get_text()

            if pl == '' or pl == '\xa0':
                continue
            
            else:
                years.append(pl)
                data[pl] = []

        # iterate thorugh each industry and append data to each year

        for i in test.find(id=page).find(class_='report_table_data').find('tbody').find_all('tr')[3:-2]:

            industry = i.find('td').get_text().strip()

            init = 0

            for row in i.find_all('td')[2:]:
                
                if row.get_text() == '\xa0':
                    continue

                if row.get_text() == 'S':
                    data[years[init]].append(np.nan)
                else:

                    data[years[init]].append(int(row.get_text().replace(',', '')))

                init += 1
                
    # convert dict to df
                
    data_df = pd.DataFrame.from_dict(data, orient='index', columns=industries)
                
    mypath = 'Counties'
    if not os.path.isdir(mypath):
        os.makedirs(mypath)

    data_df.to_csv(f"{mypath}/{county}.csv", index=True)

In [13]:
# obtain all industry names for dataframe

industries = []

for i in test.find(id='page_1').find(class_='report_table_data').find('tbody').find_all('tr')[3:-1]:
    
    industry = i.find('td').get_text().strip()
    
    if industry == '':
        continue
        
    industries.append(industry)

In [38]:
browser.quit()